In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()   
OPENAI_KEY = os.getenv("OPENAI_KEY")
# MODEL = "gpt-3.5-turbo"
MODEL = "llama2"
 

In [ ]:
# from langchain_openai.chatmodel import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
# if MODEL.startswith("gpt"):
#     model = ChatOpenAI(model_name=MODEL, openai_api_key=OPENAI_KEY)
# else:


model = Ollama(model=MODEL)
embeddings =  OllamaEmbeddings()

model.invoke("Tell me a joke.")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
# parser mostly important for the openai_gpt cause it is made only for chatting purpose.
parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke.")


In [ ]:
chain.invoke("Tell me a joke.")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Python.pdf")
pages = loader.load_and_split()


In [ ]:
pages

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the 
question, reply "I don't Know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

In [ ]:
chain.input_schema.schema()

In [ ]:
chain = prompt | model | parser

In [ ]:
chain.invoke(
    {
        "context": "We are learning about the python programming language",
        "question": "What are we learning?"
    }
)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [ ]:
retriver = vectorstore.as_retriever()

retriver.invoke("Python Program")

In [ ]:
from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriver, "question": itemgetter("question")}
| prompt 
| model
| parser
)

chain.invoke({"question": "What are Python Functions?"})

In [37]:
questions = [
    "What is print function?",
    "What does lambda function do?",
    "What is Dependency?",
    "what is built-in function in python, name a few?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({"question": question })}")
    print()

Answer: The answer to the question is: The "print" function in Python.

Question: What does lambda function do?
Answer: I can answer the question based on the provided context. In the context, a lambda function is a shorthand for defining a small anonymous function that can be used as an argument to a larger function or passed as a callback function to be called at a later time. The lambda function in this case takes two arguments, `log` and `page`, and returns a string that includes the page number and the log level.

In more detail, a lambda function is a small anonymous function that can be defined inline within a larger expression or statement. It takes one or more arguments, which are passed to the function when it is called, and returns a value. Lambda functions are often used as callback functions or as arguments to other functions. They are particularly useful in situations where a small, simple function is needed, but the function does not need to be defined as a separate modu

In [ ]:
for s in chain.stream({"question": "Explain Python Functions in detail."}):
    print(s, end="", flush=True)

In [38]:
chain.batch([{"question": q} for q in questions])

['The answer to the question is: The `print()` function in Python.',
 'The lambda function is a special type of function in Python that is defined inline, within a larger expression or statement. It is often used to create small, anonymous functions that can be used to simplify code or perform short-lived operations. The lambda function can take any number of arguments, just like a regular function, and can return a value.\n\nIn the context of the passage you provided, the lambda function is used in the `download()` function to create an anonymous function that calls the `print()` function with the downloaded file name as its argument. The lambda function is defined within the `download()` function, and it takes a single argument, `msg`, which is the message to be printed.\n\nThe use of lambda functions in this passage highlights one of their key benefits: they can be used to simplify code by eliminating the need for a separate function definition. In this case, the lambda function ser